In [1]:
import json
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm 
import torch
from torch_geometric.data import Data
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
import pandas as pd

import difflib
from sklearn.feature_extraction.text import TfidfVectorizer

import sys
sys.path.append('../AJA')
import AJA as aja

C:\Users\arnau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# récupération des données 
df_train_nodes, df_train_edges, df_test_nodes, df_test_edges = aja.get_data()

In [ ]:
# feature extraction

from sklearn.preprocessing import StandardScaler

# node

scaler = StandardScaler()
# sentence length normalized
df_train_nodes['sentence_length'] = df_train_nodes['text'].apply(lambda s: len(s.split()))
df_train_nodes['sentence_length'] = scaler.fit_transform(df_train_nodes['sentence_length'].values.reshape(-1, 1))
df_test_nodes['sentence_length'] = df_test_nodes['text'].apply(lambda s: len(s.split()))
df_test_nodes['sentence_length'] = scaler.transform(df_test_nodes['sentence_length'].values.reshape(-1, 1))

df_train_nodes['nb_occurences'] = df_train_nodes['text'].apply(lambda x: sum(x.split().count(mot) for mot in ['uh', 'um', 'okay', '<', 'ah', 'oh']))
df_train_nodes['nb_occurences'] = scaler.fit_transform(df_train_nodes['nb_occurences'].values.reshape(-1, 1))
df_test_nodes['nb_occurences'] = df_test_nodes['text'].apply(lambda x: sum(x.split().count(mot) for mot in ['uh', 'um', 'okay', '<', 'ah', 'oh']))
df_test_nodes['nb_occurences'] = scaler.transform(df_test_nodes['nb_occurences'].values.reshape(-1, 1))


df_train_nodes['nb_words_more_5'] = df_train_nodes['text'].apply(lambda x: sum(len(mot) > 5 and mot.lower() != '<vocalsound>' for mot in x.split()))
df_train_nodes['nb_words_more_5'] = scaler.fit_transform(df_train_nodes['nb_words_more_5'].values.reshape(-1, 1))
df_test_nodes['nb_words_more_5'] = df_test_nodes['text'].apply(lambda x: sum(len(mot) > 5 and mot.lower() != '<vocalsound>' for mot in x.split()))
df_test_nodes['nb_words_more_5'] = scaler.transform(df_test_nodes['nb_words_more_5'].values.reshape(-1, 1))


# speaker hot-one encoding
one_hot_encoded = pd.get_dummies(df_train_nodes['speaker_int'], prefix='speaker', dtype=int)
df_train_nodes = df_train_nodes.drop('speaker_int', axis=1)
df_train_nodes = df_train_nodes.drop('speaker_text', axis=1)
df_train_nodes = pd.concat([df_train_nodes, one_hot_encoded], axis=1)

one_hot_encoded = pd.get_dummies(df_test_nodes['speaker_int'], prefix='speaker', dtype=int)
df_test_nodes = df_test_nodes.drop('speaker_int', axis=1)
df_test_nodes = df_test_nodes.drop('speaker_text', axis=1)
df_test_nodes = pd.concat([df_test_nodes, one_hot_encoded], axis=1)

# edge

new_df = pd.DataFrame({
        'transcription': df_train_edges['transcription'],
        'start': df_train_edges['end'],
        'end': df_train_edges['start'],
        'type_int': 16 + df_train_edges['type_int'],
        'type_text': df_train_edges['type_text'] + "_reverse"
    })
df_train_edges = pd.concat([df_train_edges, new_df], ignore_index=True)

new_df = pd.DataFrame({
        'transcription': df_test_edges['transcription'],
        'start': df_test_edges['end'],
        'end': df_test_edges['start'],
        'type_int': 16 + df_test_edges['type_int'],
        'type_text': df_test_edges['type_text'] + "_reverse"
    })
df_test_edges = pd.concat([df_test_edges, new_df], ignore_index=True)

In [24]:
from sklearn.preprocessing import StandardScaler

def calculate_similarity(text, neighbors, df):
    max_similarity = 0.0
    if isinstance(neighbors, list):
        for neighbor in neighbors:
            if neighbor != []:
                if isinstance(neighbor, int):
                    neighbor_index = neighbor
                else:
                    neighbor_index = neighbor[0]
                neighbor_text = df.iloc[neighbor_index]['text']
                matcher = difflib.SequenceMatcher(None, text.split(), neighbor_text.split())
                similarity = matcher.ratio()
                max_similarity = max(max_similarity, similarity)
    elif isinstance(neighbors, int):
        # Handle the case where 'neighbors' is an integer directly
        neighbor_index = neighbors
        neighbor_text = df.iloc[neighbor_index]['text']
        matcher = difflib.SequenceMatcher(None, text.split(), neighbor_text.split())
        max_similarity = matcher.ratio()
    return max_similarity

def f(df_nodes, df_edges):

    df = df_nodes

    scaler = StandardScaler()

    # sentence length normalized
    df['sentence_length'] = df['text'].apply(lambda s: len(s.split()))
    df['sentence_length'] = scaler.fit_transform(df['sentence_length'].values.reshape(-1, 1))

    # speaker hot-one encoding
    one_hot_encoded = pd.get_dummies(df['speaker_int'], prefix='speaker')
    df = df.drop('speaker_int', axis=1)
    df = df.drop('speaker_text', axis=1)
    df = pd.concat([df, one_hot_encoded], axis=1)
    
    # TFIDF
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])

    # Somme des Scores TF-IDF par phrase
    df['tfidf_sum'] = tfidf_matrix.sum(axis=1)

    # Score TF-IDF maximum par phrase
    df['tfidf_max'] = tfidf_matrix.max(axis=1).toarray().flatten()

    # Score TF-IDF minimum par phrase
    df['tfidf_min'] = tfidf_matrix.min(aixs=1).toarray().flatten()
    
    #Text analysis
    df['nb_occurences'] = df['text'].apply(lambda x: sum(x.split().count(mot) for mot in ['uh', 'um', 'okay', '<', 'ah', 'oh']))
    df['nb_occurences'] = scaler.fit_transform(df['nb_occurences'].values.reshape(-1,1))
    
    df['nb_words_more_5'] = df['text'].apply(lambda x: sum(len(mot) > 5 and mot.lower() != '<vocalsound>' for mot in x.split()))
    df['nb_words_more_5'] = scaler.fit_transform(df['nb_words_more_5'].values.reshape(-1,1))
    
    # Calcul de la colonne 'neighb'
    df_edges_grouped = df_edges.groupby(['transcription', 'start'])['end'].apply(list).reset_index()
    df = pd.merge(df, df_edges_grouped, how='left', left_on=['transcription', 'line'], right_on=['transcription', 'start'])
    df = df.rename(columns={'end': 'neighb'})

    # Calcul de la colonne 'similarities'
    df['similarities'] = df.apply(lambda row: calculate_similarity(row['text'], row['neighb'],df), axis=1)
    df['similarities'] = scaler.fit_transform(df['similarities'].values.reshape(-1,1))

    df = df.drop(['text','start', 'neighb'], axis=1)
                    
    return df

def g(df):
    new_df = pd.DataFrame({
        'transcription': df['transcription'],
        'start': df['end'],
        'end': df['start'],
        'type_int': 16 + df['type_int'],
        'type_text': df['type_text'] + "_reverse"
    })
    result_df = pd.concat([df, new_df], ignore_index=True)
    return result_df

In [25]:
train_graphs, validation_graphs, test_graphs = aja.get_graphs(f, g, 0.2)

TypeError: _minmax_mixin.min() got an unexpected keyword argument 'aixs'

In [12]:
class MultiChannelsGCN(torch.nn.Module):
    
    def __init__(self, channels, input_dim, post_conv_dim, output_dim, identity=False):
        super(MultiChannelsGCN, self).__init__()
        self.identity = identity
        self.channels = channels
        self.input_dim = input_dim
        self.post_conv_dim = post_conv_dim
        self.output_dim = output_dim
        self.GCN = nn.ModuleList([GCNConv(input_dim, post_conv_dim) for _ in range(channels)])
        if identity:
            self.dense = nn.Linear(post_conv_dim * (channels + 1), output_dim)
            self.denseID = nn.Linear(input_dim, post_conv_dim)
        else:
            self.dense = nn.Linear(post_conv_dim * channels, output_dim)

    def forward(self, nodes, edges):
        X = []
        for k in range(self.channels):
            if len(edges[k]) == 0:
                x = torch.zeros(nodes.shape[0], self.post_conv_dim)
            else:
                x = F.relu(self.GCN[k](nodes, edges[k]))
            X.append(x)
        if self.identity:
            X.append(F.relu(self.denseID(nodes)))
        concat = torch.cat(X, dim=1)
        return F.relu(self.dense(concat))

In [13]:
# on définie son plus beau modèle

class NodeClassifier(torch.nn.Module):
    def __init__(self, channels, input_dim):
        super(NodeClassifier, self).__init__()
        self.threshold = 0.5
        self.GCN1 = MultiChannelsGCN(channels, input_dim, 50, 20, identity=True)
        self.dropout = nn.Dropout(0.3)  # Couche de dropout 
        self.dense1 = nn.Linear(20,1)

    def forward(self, data):
        nodes, edges = data.x, data.edge_index
        
        x = self.GCN1(nodes, edges)
        x = self.dense1(x)
        return x

    def set_threshold(self, t):
        self.threshold = t

    def predict(self, graph): #version sans threshold
        self.eval()
        with torch.no_grad():
            logits = self.forward(graph)
        return np.array((logits > 0.5).int()).flatten()
    
    '''def predict(self, graph): #version avec threshold
        self.eval()
        with torch.no_grad():
            logits = self.forward(graph)
            predictions = torch.sigmoid(logits)
        return np.array((predictions > self.threshold).int()).flatten()'''

In [14]:
# Charger l'extension autoreload
%load_ext autoreload

# Configurer autoreload pour recharger tous les modules avant l'exécution de chaque cellule
%autoreload 2

In [15]:
N_features = train_graphs['ES2002b'].x.shape[1]
N_features

395

In [16]:
# Move the instantiation of the model outside the training loop
model = NodeClassifier(32, N_features)
#model.set_threshold(0.59)
model_name='test'

# Move the model and data to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Use DataLoader to create batches of data
'''train_loader = DataLoader(list(train_graphs.values()), batch_size=1, shuffle=True)
N_train = len(train_loader)
'''
train_loader = DataLoader(list(train_graphs.values()) + list(validation_graphs.values()), batch_size=1, shuffle=True)
N_train = len(train_loader)

validation_loader = DataLoader(list(validation_graphs.values()), batch_size=1, shuffle=False)
N_validation = len(validation_loader)

# Define the loss function and optimizer
alpha = 0.2
gamma = 5
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor((1 - alpha) / alpha), reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

from sklearn.metrics import f1_score

# Train the model
best_f1_score = 0
for epoch in range(5):
    print('- Epoch', f'{epoch:03d}', '-')

    # training
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data).squeeze()
        loss = criterion(output, data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    average_loss = total_loss / N_train
    print('Loss:', f'{average_loss:.4f}')    

    # Evaluate the model on the training set

    model.eval()
    f1_moyen_train = 0
    for data in train_loader:
        data = data.to(device)
        y_pred = model.predict(data)
        y_true = data.y.cpu().numpy()
        f1 = f1_score(y_true, y_pred)
        f1_moyen_train += f1
    f1_moyen_train /= N_train
    print('F1 train:', f1_moyen_train)

    # Evaluate the model on the validation set
    model.eval()
    f1_moyen_valid = 0
    for data in validation_loader:
        data = data.to(device)
        y_pred = model.predict(data)
        y_true = data.y.cpu().numpy()
        f1 = f1_score(y_true, y_pred)
        f1_moyen_valid += f1
    f1_moyen_valid /= N_validation
    print('F1 valid:', f1_moyen_valid)

    # callbacks ou autre
    if f1_moyen_valid > best_f1_score:
        torch.save(model.state_dict(), "training_states/" + model_name + "-best.pth")
    else:
        optimizer.param_groups[0]['lr'] /= 2
        print('Learning rate reduced to:', optimizer.param_groups[0]['lr'])
    print('')

model.load_state_dict(torch.load("training_states/" + model_name + "-best.pth"))

- Epoch 000 -
Loss: 0.7606
F1 train: 0.5712341649543731
F1 valid: 0.5911355316637193

- Epoch 001 -
Loss: 0.6656
F1 train: 0.5755088900230775
F1 valid: 0.5894371469242895

- Epoch 002 -
Loss: 0.6760
F1 train: 0.5842017459254026
F1 valid: 0.6017656930747759

- Epoch 003 -
Loss: 0.6642
F1 train: 0.5804868186546803
F1 valid: 0.5912627030305966

- Epoch 004 -
Loss: 0.6536
F1 train: 0.5969166831134869
F1 valid: 0.6124502842859502

- Epoch 005 -
Loss: 0.6325
F1 train: 0.5854816708924322
F1 valid: 0.6029318579571085

- Epoch 006 -
Loss: 0.6533
F1 train: 0.592810344629942
F1 valid: 0.611195457678265



<All keys matched successfully>

In [123]:
T = np.linspace(0.4,0.6,10)
f1s = []
for t in T:
    model.set_threshold(t)
    f1_valid = aja.f1_score_moyen(model, validation_graphs)
    f1s.append(f1_valid)

plt.plot(T, f1s)
plt.xlabel('Threshold')
plt.ylabel('F1-score')
plt.show()

In [137]:
# quand on est content on fait une submission !
#model.set_threshold(0.59) #on choisit le threshold adapté
aja.make_test_csv_submission(model, test_graphs, 'features_tf_idf')